In [1]:
import os

from omegaconf import DictConfig, OmegaConf
from hydra import initialize, initialize_config_module, initialize_config_dir, compose

from datasets.squidle_data import SquidleData
from datasets.squidle_connection import SquidleAnnotator, SquidleConnection
from sqapi.api import SQAPI


#### 0. Setup config

In [2]:
### Config and squidle access setup
config = "squidle_hand_target.yaml"
abs_config_dir=os.path.abspath("../config/dataset")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    opt = compose(config_name=config)
    #print(OmegaConf.to_yaml(opt, resolve=True))
#print(opt)
OmegaConf.set_struct(opt, False)

dataset = SquidleData(opt, semi_supervised=True)
sq_annotator = SquidleAnnotator(api_key=opt.annotate_api_token, host=opt.url, 
                                label_map_file="./datasets/example_coco.json")
sq_annotator = SquidleAnnotator(api_key=opt.api_token, host=opt.url,
                                label_map_file="./datasets/example_coco.json")
squidle_connection = SquidleConnection(sqapi=sq_annotator.sqapi)

/home/heather/.conda/envs/py310_torch/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'squidle_hand_target.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


#### 1. Get annotations


Red Cup sponge not in [5862, 5863, 12745, 12750, 12747, 12739]

Counter({13249: 919, 5861: 121, 12749: 116, 12748: 109, 12737: 100, 12751: 99, 10567: 36, 10566: 26, 7933: 24, 12339: 23, 10570: 19, 7914: 18, 7925: 18, 5853: 16, 466: 14, 12740: 14, 5852: 13, 7927: 12, 8126: 12, 7932: 11, 11625: 10, 12444: 10, 11637: 9, 7987: 8, 4003: 7, 7978: 7, 10546: 6, 10862: 6, 10880: 6, 11609: 6, 7940: 5, 8048: 5, 7931: 4, 8127: 4, 9472: 4, 10547: 4, 3418: 3, 5869: 3, 7939: 3, 7984: 3, 10870: 3, 3988: 2, 7770: 2, 10885: 2, 11186: 2, 442: 1, 2133: 1, 7929: 1, 7937: 1, 7938: 1, 7941: 1, 7942: 1, 8100: 1, 8123: 1, 10548: 1, 10553: 1})
Counter({7111: 1238, 13288: 540, 7104: 77})

In [39]:
label_ids = list(dataset.opt.squidle_mapping)
#label_ids = [13450]
annotation_set_ids = list(dataset.opt.annotation_set_ids) if dataset.opt.annotation_set_ids is not None else []
#annotation_set_ids = [5862, 5863, 12745, 12750, 12747, 12739] # train
exclude_annotation_set_ids = dataset.opt.exclude_annotation_set_ids if dataset.opt.exclude_annotation_set_ids is not None else []
#annotation_set_ids = [10827, 10829] # test
#annotation_set_ids = []

In [72]:
annotation_list = dataset.squidle_connection.get_all_annotations_from_set(annotation_set_ids, label_ids=label_ids,
                                                              include_annotation_sets=True,
                                                              exclude_annotation_sets=list(set(dataset.opt.needs_review_annotation_set_ids) |
                                                                          set(exclude_annotation_set_ids)),
                                                              results_per_page=500)

HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"label","op":"has","val":{"name":"id","op":"in","val":[13450]}},{"name":"annotation_set_id","op":"in","val":[13372,13373]},{"name":"needs_review","op":"eq","val":"False"},{"name":"point","op":"has","val":{"name":"has_xy","op":"eq","val":true}}]}&page=1&results_per_page=500 ... ✅ | 200 | Done in 7.23 s
Extracting 1 pages with 284 records.
HTTP-GET: https://squidle.org/api/annotation_set/13372 ... ✅ | 200 | Done in 0.72 s
HTTP-GET: https://squidle.org/api/annotation_set/13373 ... ✅ | 200 | Done in 0.69 s
Counter({13373: 242, 13372: 42})
[]
Counter({13450: 284})


In [24]:
asi = list(set(dataset.opt.needs_review_annotation_set_ids) |
           set(exclude_annotation_set_ids))

In [ ]:
print(dataset.squidle_connection.sqapi.get(f"/api/annotation_set/13308").execute().json())



In [35]:
# Run this if you're excluding annotation sets.
annotation_list +=  dataset.squidle_connection.get_all_annotations_from_set(list(dataset.opt.needs_review_annotation_set_ids),
                                                                       label_ids=label_ids,
                                                                       include_annotation_sets=True,
                                                                       needs_review=True,
                                                                       results_per_page=200)

HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"label","op":"has","val":{"name":"id","op":"in","val":[13450]}},{"name":"annotation_set_id","op":"in","val":[13142,13150,13156,13144,13146,13147,13148,13149,13152,13153,13157,13159,13161,13163,13180,13205,13207,13208,13209,13210,13240,13252,13264,13285,13286,13287,13288,13289,13308,13309,13310,13311,13312,13319,13373,13377,13378,13379,13382,13383,13385,13386,13387,13388,13389]},{"name":"needs_review","op":"eq","val":"True"},{"name":"point","op":"has","val":{"name":"has_xy","op":"eq","val":true}}]}&page=1&results_per_page=200 ... ✅ | 200 | Done in 6.62 s
Extracting 1 pages with 91 records.
HTTP-GET: https://squidle.org/api/annotation_set/13142 ... ✅ | 200 | Done in 0.85 s
HTTP-GET: https://squidle.org/api/annotation_set/13144 ... ✅ | 200 | Done in 0.84 s
HTTP-GET: https://squidle.org/api/annotation_set/13146 ... ✅ | 200 | Done in 0.94 s
HTTP-GET: https://squidle.org/api/annotation_set/13148 ... ✅ | 200 | Done in 0.91 s
H

##### Ensure the training data (semi/excluding test/val annotation_sets) does not have any media from them.
  - 10820 # 4 handfish
  - 10827 # 4 handfish
  - 10829 # 6 handfish
  - 10818 # 1 handfish
  - 10819 # 1 handfish


In [41]:
excluded_media_collection_ids = []
excluded_media = []
#annotation_set_ids = [10829, 10819, 10818, 10820, 10827]
annotation_set_ids = list(set([a['annotation_set_id'] for a in annotation_list]))

In [42]:
media_id_dict = {}
#annotation_set_ids = [8103]
for asi in annotation_set_ids:
    media_collection_id = dataset.squidle_connection.sqapi.get(f"/api/annotation_set/{asi}").execute().json()
    media_collection_id = media_collection_id['media_collection']['id']
    excluded_media_collection_ids.append(media_collection_id)
    page = 1
    request = sq_annotator.sqapi.get("/api/media",
                                     page=page, results_per_page=1000)
    request.filter(name="media_collection_media", op="any",
                   val={'name': "media_collection_id", 'op': 'eq', 'val': media_collection_id})
    result = request.execute().json()
    for m in result['objects']:
        media_id_dict[m['id']] = m
    excluded_media += [m['id'] for m in result['objects']]
    print(f"Total pages {result['total_pages']}")
    while page < result['total_pages']:
        page += 1
        request = sq_annotator.sqapi.get("/api/media",
                                         page=page, results_per_page=1000)
        request.filter(name="media_collection_media", op="any",
                       val={'name': "media_collection_id", 'op': 'eq', 'val': media_collection_id})
        result = request.execute().json()
        for m in result['objects']:
            media_id_dict[m['id']] = m        
        excluded_media += [m['id'] for m in result['objects']]        
media_ids = list(set([a['point']['media_id'] for a in annotation_list]))

HTTP-GET: https://squidle.org/api/annotation_set/13372 ... ✅ | 200 | Done in 0.74 s
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":11666}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.12 s
Total pages 1
HTTP-GET: https://squidle.org/api/annotation_set/13373 ... ✅ | 200 | Done in 0.70 s
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":11667}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.23 s
Total pages 1


In [43]:
# Get media_items for annotation_list
page = 1
request = sq_annotator.sqapi.get("/api/media",
                                 page=page, results_per_page=1000)
request.filter(name="annotations", op="any",
               val={'name': "id", 'op': 'in', 'val': list(set([a['point']['id'] for a in annotation_list]))})
result = request.execute().json()
media_id_dict = {m['id']: m for m in result['objects']}    

HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"annotations","op":"any","val":{"name":"id","op":"in","val":[8859136,8859137,8859138,8859139,8859140,8859141,8859142,8859143,8859144,8859145,8859146,8859147,8859148,8859149,8859150,8859151,8859152,8859153,8859154,8859155,8859156,8859157,8859158,8859159,8859160,8859161,8859162,8859163,8859164,8859165,8859166,8859167,8859168,8859169,8859170,8859171,8859172,8859173,8859174,8859175,8859176,8859177,8859178,8859179,8859180,8859181,8859182,8859183,8859184,8859185,8859186,8859187,8859188,8859189,8859190,8859191,8859192,8859193,8859194,8859195,8859196,8859197,8859198,8859199,8859200,8859201,8859202,8859203,8859204,8859205,8859206,8859207,8859208,8859209,8859210,8859211,8859212,8859213,8859214,8859215,8859216,8859217,8859218,8859219,8859220,8859221,8859222,8859223,8859224,8859225,8859226,8859227,8859228,8858745,8858746,8858747,8858748,8858749,8858750,8858751,8858752,8858753,8858754,8858755,8858756,8858757,8858758,8858759,8858760,88587

In [44]:
from collections import defaultdict
annotation_media_id_dict = defaultdict(list)
for a in annotation_list:
    media_id = a['point']['media_id']
    annotation_media_id_dict[media_id].append(a)
    if len(annotation_media_id_dict[media_id]) > 1:
        print(media_id)

3258836
3256172


In [54]:
from collections import Counter
deployment_counter = []
for a_media_id in annotation_media_id_dict.keys():
    d_id = media_id_dict[a_media_id]['deployment_id']
    d_key = media_id_dict[a_media_id]['deployment']['key']
    deployment_counter.append(f"{d_id}_{d_key}")

deployment_counter = Counter(deployment_counter)
for d in sorted(deployment_counter):
    print(d, deployment_counter[d])


11668_r20210614_000137_SS04_TF_NPZ_07 31
11669_r20210614_034824_SS05_TF_NPZ_01 2
11670_r20210614_062740_SS06_TF_NPZ_02 2
11671_r20210614_222648_SS07_TF_NPZ_06 6
11673_r20210615_024126_SS09_TF_NPZ_03 151
11674_r20210615_054656_SS10_TF_NPZ_08 3
11675_r20210615_222253_SS11_TF_Ref_N_1 4
11676_r20210616_004810_SS12_TF_Ref_N_1_second_half 1
11682_r20210617_033152_SS18_TF_NPZ_04_reversed 2
11683_r20210617_050102_SS19_TF_NPZ_04_reversed_second_half 1
13147_r20220807_220408_NG02_huon_08_deep_out1_repeat 10
13149_r20220808_022634_NG04_huon_13_deep_in2 1
13156_r20220905_215914_NG18_huon_13_deep_in2_take2 5
13157_r20220906_003021_NG19_huon_1_new 1
13158_r20220906_045539_NG21_huon_2_new_take2 4
13159_r20220906_210124_NG22_huon_3_new 2
13160_r20220907_002130_NG23_huon_4_new 4
14497_r20230311_204310_NG25_m6_freycinet_break_3 7
14498_r20230311_234059_NG26_m7_freycinet_break_4 5
303_r20090615_225253_huon_mpa_16 4
312_r20100604_061515_huon_13_deep_in2 27
404_r20140618_223233_tas_18_huon_deep_in1 3
407_r

LOCKED Test15 - 42 images from deployment 11682 and ahead (ie r20210617_033152_SS18_TF_NPZ_04_reversed 2)
train85 - everything before 20210617 - 242 annotations



In [81]:


train, test = [], []
for a in annotation_list_orig:
    media_id = a['point']['media_id']
    if media_id_dict[media_id]['deployment_id'] >= 11676:
        test.append(a)
    else:
        train.append(a)
        


In [58]:
clean_annotation_list = []
for a in annotation_list:
    media_id = a['point']['media_id']
    if media_id not in excluded_media:
        clean_annotation_list.append(a)
    else:
        print(f"Removing {media_id}")
        
annotation_list = clean_annotation_list
        

Removing 8119081
Removing 8128289
Removing 7670737
Removing 7746890
Removing 7726354
Removing 7769497
Removing 8128291
Removing 8119078
Removing 8119080
Removing 8119083
Removing 8119082
Removing 8119079
Removing 8119077
Removing 8128287
Removing 8128290
Removing 8128288
Removing 6972834
Removing 6976242
Removing 6973050
Removing 7736233
Removing 7744604
Removing 7742018
Removing 7741868
Removing 7754719
Removing 7751269
Removing 7766389
Removing 7655043
Removing 7652495
Removing 7652492
Removing 7655042
Removing 7655044
Removing 7652493
Removing 7769935
Removing 7727842
Removing 7763575
Removing 7727314
Removing 7652494
Removing 7655041
Removing 7727560
Removing 7726918
Removing 7652491
Removing 7655045
Removing 6907557
Removing 6907686
Removing 6907542
Removing 6902549
Removing 6902546
Removing 6910661
Removing 6902548
Removing 6907544
Removing 6902553
Removing 6907688
Removing 6908366
Removing 6908539
Removing 6907555
Removing 6902114
Removing 6902552
Removing 6909501
Removing 69074

#### 2. Create media_collection and annotation_set

##### Creating random selsections of training data
Create 4 * 50 random selections plus 1 X 41 images for validation

In [115]:
import random

def findElements(lst1, lst2):
    return list(np.array(lst1)[lst2])
selection = [i for i in range(len(train))]
random.shuffle(selection)

In [116]:
val = selection[0:41]
train_1 = selection[41:91]
train_2 = selection[91:141]
train_3 = selection[141:191]
train_4 = selection[191:241]

In [119]:
set(val).intersection(set(train_3))

set()

In [144]:
#original_query_annotation_list = annotation_list
annotation_list = findElements(train, train_4)

In [56]:
description = f"Media and annotations for deployments including and before 11671_r20210614_222648_SS07_TF_NPZ_06 6  from annotation sets {set([a['annotation_set_id'] for a in annotation_list])}"
description = "Media and annotations for all unique handfish detections by either human or AI."
#description = f"Random selection of n=50 handfish annotations v4 from Train 85pc dataset (13373) for training"
name = "Handfish Detections - Human and AI generated"
group_id = 358

In [57]:
media_collection_id = sq_annotator.create_media_collection(name=name, description=description)
print(media_collection_id)

HTTP-POST: https://squidle.org/api/media_collection ... ✅ | 201 | Done in 0.18 s
13053


##### Add media to collection

In [59]:
# ADDING MEDIA to collection
media_ids = list(set([a['point']['media_id'] for a in objs]))
sq_annotator.add_media_to_collection(media_collection_id, media_ids)
print("Finished!!!")

HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":13053}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.10 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6903820 ... ✅ | 200 | Done in 0.10 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6903822 ... ✅ | 200 | Done in 0.10 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910020 ... ✅ | 200 | Done in 0.10 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910021 ... ✅ | 200 | Done in 0.09 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910022 ... ✅ | 200 | Done in 0.12 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910023 ... ✅ | 200 | Done in 0.09 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910024 ... ✅ | 200 | Done in 0.10 s
HTTP-POST: https://squidle.org/api/media_collection/13053/media/6910025 ...

In [ ]:
# ONEOFF - Used to add additional media items for annotation for Helico fish
media_collection_id=11646
media_ids_to_add = [7139855, 7139911, 7139287, 7139856, 7143660, 7144053, 7140201, 7142111, 7143767, 
             7142952, 7143686, 3267023, 3266573, 3266744, 3251543, 3251665, 3251754, 3253943, 
             3256361, 3265129, 3268157, 3264603, 7055817, 7056643, 7057201, 7057433, 7057206,
             7062009, 7062048, 7081110, 7081604, 7087512, 7087212, 7090604, 7092440, 7087588, 7094478,
                    7087924, 7108698]
sq_annotator.add_media_to_collection(media_collection_id, media_ids_to_add)

In [ ]:
print(set(media_ids).intersection(set(excluded_media)))

##### Create annotation set

In [5]:
media_collection_id = 11592 #Handfish Train Dataset

* New annotation set (like other annotation sets?)
* Add annotations

In [60]:
# Get label_scheme_id
label_scheme_id = 7
label_scheme_ids = list(set([a['label']['label_scheme_id'] for a in annotation_list]))
print(label_scheme_ids)

[7]


In [61]:
annotation_set_id = sq_annotator.create_annotation_set(name, description, media_collection_id, label_scheme_id, group_id=group_id)
print(annotation_set_id)

HTTP-POST: https://squidle.org/api/annotation_set ... ✅ | 201 | Done in 0.78 s
HTTP-POST: https://squidle.org/api/annotation_set/15800/group/358 ... ✅ | 200 | Done in 0.10 s
15800


#### 3. Add annotations


In [ ]:
# Use this if you're not creating a new media_collection and annotation set
annotation_set_id = annotation_list[0]['annotation_set_id']
media_collection_id = dataset.squidle_connection.sqapi.get(f"/api/annotation_set/{annotation_set_id}").execute().json()
#media_collection_id = media_collection_id['media_collection']['id']



In [21]:
print(f"Using media_collection {media_collection_id} and annotation_set {annotation_set_id}")

Using media_collection 13052 and annotation_set 15799


In [ ]:
#media_collection_id = 11598
#annotation_set_id = Nonez

In [62]:
sq_annotator.add_annotations_to_annotation_set(annotation_set_id, media_collection_id, annotation_list,
                                               label_scheme_id=label_scheme_id)
print("Finished!!!")

HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":13053}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.45 s
Downloaded image: https://s3-ap-southeast-2.amazonaws.com/imos-data/IMOS/AUV/auv_viewer_data/images/Tasmania201006/r20100604_061515_huon_13_deep_in2/full_res/PR_20100604_072454_294_LC16.jpg in 0.27043962478637695 s...
HTTP-POST: https://squidle.org/api/point ... ✅ | 200 | Done in 0.10 s
Downloaded image: https://data.acfr.usyd.edu.au/marine/TasFracture202106/r20210614_062740_SS06_TF_NPZ_02/full_res/PR_20210614_075727_549_LC16.jpg in 1.6660637855529785 s...
HTTP-POST: https://squidle.org/api/point ... ✅ | 200 | Done in 0.10 s
Downloaded image: https://data.acfr.usyd.edu.au/marine/TasFracture202106/r20210614_062740_SS06_TF_NPZ_02/full_res/PR_20210614_075729_550_LC16.jpg in 0.8876962661743164 s...
HTTP-POST: https://squidle.org/api/point ... ✅ | 200 | Done in 0.08 s
Download

##### Steps to add annotations

In [44]:
import numpy as np
import torch
from sqapi.media import SQMediaObject
from segment_anything import sam_model_registry, SamPredictor
import cv2


In [45]:
sam_checkpoint = "../../segmentAnything/checkpoints/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

In [ ]:
request = sq_annotator.sqapi.get("/api/media",
                         page=1, results_per_page=5000)
request.filter(name="media_collection_media", op="any",
               val={'name': "media_collection_id", 'op': 'eq', 'val': media_collection_id})
media_data = request.execute().json()['objects']
media_lookup = {m['id']: m for m in media_data}

# Create a simple code lookup based on annotation labels.
# todo: make this robust to different label schemes
label_set = list(
    set([a['label']['id'] for a in annotation_list]))
sq_annotator.code2label = {l: {'id': l} for l in label_set}


In [ ]:
annotation_media_id_dict[7769497][0]

In [ ]:
for a in annotation_list:
    # Get the point and media_obj from the annotation
    annotation = annotation_media_id_dict[a['point']['media_id']][0]
    point = annotation['point']
    get_contours = False
    if annotation['annotation_set_id'] != annotation_set_id:
        get_contours = True
    else:
        if 'data' in point and 'polygon' in point['data']:
            polygon = point['data']['polygon']
            if len(polygon) == 4:
                get_contours = True
    if get_contours:
        m = media_lookup[point['media_id']]
        media_url = m.get('path_best')
        media_type = m.get("media_type", {}).get("name")
        mediaobj = SQMediaObject(media_url, media_type=media_type, media_id=m.get('id'))
        if not mediaobj.is_processed:
            orig_image = mediaobj.data()
        width = mediaobj.width
        height = mediaobj.height
        x = int(point.get('x') * width)
        y = int(point.get('y') * height)
        label = annotation['label']['id']

        # SAM
        predictor = SamPredictor(sam)
        predictor.set_image(orig_image)
        input_point = np.array([[x, y]])
        input_box = None
        if 'data' in point and 'polygon' in point['data']:
            polygon = point['data']['polygon']
            if len(polygon) == 4:
                input_box = dataset.get_bbox_in_pixels(point.get('x'), point.get('y'), polygon, width, height, buffer=0.01)
                input_box[2] = input_box[0] + input_box[2]
                input_box[3] = input_box[1] + input_box[3]
                input_box = np.array(input_box)

        input_label = np.array([label])
        masks, _, _ = predictor.predict(
            point_coords=input_point,
            point_labels=input_label,
            box=input_box,
            multimask_output=False,
        )

        imgray = masks[0, :, :].astype(np.uint8)*255
        contours, _ = cv2.findContours(imgray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1) #, cv2.CHAIN_APPROX_TC89_KCOS)
        polygons = []

        polygons = []
        for obj in contours:
            for c in obj:
                coords = []
                coords.append(int(c[0][0]))
                coords.append(int(c[0][1]))
                polygons.append(coords)


        likelihood = annotation.get('likelihood', 1.0)
        likelihood = 1.0 if likelihood is None else likelihood

        # Create a new point in the annotation set and add it.
        point_data = sq_annotator.create_annotation_label_point_px(annotation['label']['id'],
                                                  likelihood=likelihood, 
                                                  comment=annotation['comment'],
                                                  row=x, col=y, width=width, height=height,
                                                  polygon=polygons,
                                                  t=point['t'])
        point_data['annotation_set_id'] = annotation_set_id
        point_data['media_id'] = mediaobj.id
        if isinstance(point_data.get('annotation_label'), dict):
            point_data['annotation_label']['annotation_set_id'] = annotation_set_id

        # Create and post point dictionary with annotation_set, media and label data.
        if annotation['annotation_set_id'] == annotation_set_id:
            # Add contour polygon if there isn't one.
            print(polygons)
            print(input_box)
            print(annotation['id'], annotation['point']['media_id'], annotation['point']['id'], annotation['comment'])
            last_annotation = annotation
            
        else:
            sq_annotator.sqapi.post("/api/point", json_data=point_data).execute()
            print(point_data)


In [ ]:
for annotation in annotation_list:
    # Get the point and media_obj from the annotation
    point = annotation['point']
    get_contours = False
    if annotation['annotation_set_id'] != annotation_set_id:
        get_contours = True
    else:
        if 'data' in point and 'polygon' in point['data']:
            polygon = point['data']['polygon']
            if len(polygon) == 4:
                get_contours = True
    if get_contours:
        m = media_lookup[point['media_id']]
        media_url = m.get('path_best')
        media_type = m.get("media_type", {}).get("name")
        mediaobj = SQMediaObject(media_url, media_type=media_type, media_id=m.get('id'))
        if not mediaobj.is_processed:
            orig_image = mediaobj.data()
        width = mediaobj.width
        height = mediaobj.height
        x = int(point.get('x') * width)
        y = int(point.get('y') * height)
        label = annotation['label']['id']


        likelihood = annotation.get('likelihood', 1.0)
        likelihood = 1.0 if likelihood is None else likelihood

        # Create a new point in the annotation set and add it.
        point_data = sq_annotator.create_annotation_label_point_px(annotation['label']['id'],
                                                                   likelihood=likelihood,
                                                                   comment=annotation['comment'],
                                                                   row=x, col=y, width=width, height=height,
                                                                   polygon=polygon,
                                                                   t=point['t'])
        point_data['annotation_set_id'] = annotation_set_id
        point_data['media_id'] = mediaobj.id
        if isinstance(point_data.get('annotation_label'), dict):
            point_data['annotation_label']['annotation_set_id'] = annotation_set_id

        # Create and post point dictionary with annotation_set, media and label data.
        if annotation['annotation_set_id'] == annotation_set_id:
            # Add contour polygon if there isn't one.
            print(polygons)
            print(input_box)
            print(annotation['id'], annotation['point']['media_id'], annotation['point']['id'], annotation['comment'])
            last_annotation = annotation
            response = sq_annotator.sqapi.patch(f"/api/annotation/8875137", json_data={'pixels': point_data['pixels']}).execute().json()
            print(response)

        else:
            sq_annotator.sqapi.post("/api/point", json_data=point_data).execute()
            print(point_data)

In [ ]:
print(dataset.squidle_connection.sqapi.get("api/annotation/8877749").execute().json())

In [ ]:
response = sq_annotator.sqapi.patch(f"/api/annotation/8875137", json_data={'pixels': point_data['pixels']}).execute().json()
print(response)
response = sq_annotator.sqapi.patch(f"/api/annotation/8875137", json_data={'pixels': point_data['pixels']}).execute().json()
print(response)

In [ ]:
response = sq_annotator.sqapi.post(f"/api/annotation", json_data=point_data).execute().json()
print(response)

In [23]:
scaled_polygon = []
for c in point_data['pixels']['polygon']:
    scaled_coord = []
    scaled_coord.append(c[0]/point_data['pixels']['width'])
    scaled_coord.append(c[1]/point_data['pixels']['height'])
    scaled_polygon.append(scaled_coord)

In [40]:
del point_data['annotation_label']
del point_data['t']

In [ ]:
response = sq_annotator.sqapi.get(f"/api/point/8720499").execute()
print(response.json())

In [ ]:
print(point_data)

In [ ]:
response = sq_annotator.sqapi.patch(f"/api/point/8717893", json_data={'polygon': scaled_polygon}).execute().json()
print(response)

In [ ]:
print(scaled_polygon)

## Get media and pose details for plotting


In [ ]:
request = dataset.sqapi.get("/api/media",
                         page=1, results_per_page=500)
request.filter(name="media_collection_media", op="any", val={'name': "media_collection_id", 'op': 'eq', 'val': media_collection_id})
media_data = request.execute().json()['objects']


In [ ]:
for md in media_data:
    request = dataset.sqapi.get("/api/pose",
                                page=1, results_per_page=500)
    request.filter(name="media_id", op="eq", val=md['id'])
    results = request.execute().json()
    md['pose'] = results['objects']

In [ ]:
import pandas as pd
import seaborn as sns


In [ ]:
output = []
for md in media_data:
    pose = md['pose'][0]
    output.append([md['id'],pose['campaign']['key'], pose['deployment']['id'],pose['dep'],pose['lat'],pose['lon']])
df = pd.DataFrame(output, columns=['id','campaign', 'deployment', 'dep', 'lat', 'lon'])

In [ ]:
sns.swarmplot(data=df, x="lon", y="lat", hue="campaign", palette="bright")


## Extract bot handfish annotations

### Query to retrieve annotations for user, label and needs_review flag


In [3]:
# Query
endpoint = '/api/annotation'
filter_list1 = [dict(name="user", op="has", val=dict(name="username", op="in", val=["acfrbot", "hdoi5324"])),
               dict(name="label", op="has", val=dict(name="id", op="eq", val="13450")),
               dict(name="needs_review", op="eq", val=True)]
filter_list3 = [dict(name="user", op="has", val=dict(name="username", op="in", val=["hdoi5324"])),
                dict(name="label", op="has", val=dict(name="id", op="eq", val="13450")),
                dict(name="needs_review", op="eq", val=False),
                dict(name="annotation_set_id", op="in", val=[14174])]
filter_list2 = [dict(name="user", op="has", val=dict(name="username", op="not_in", val=["acfrbot", "hdoi5324"])),
                dict(name="label", op="has", val=dict(name="id", op="eq", val="13450"))]
objs = []

objs = squidle_connection.recursive_get(endpoint, filter_list1)
objs += squidle_connection.recursive_get(endpoint, filter_list2)
objs += squidle_connection.recursive_get(endpoint, filter_list3)

HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"user","op":"has","val":{"name":"username","op":"in","val":["acfrbot","hdoi5324"]}},{"name":"label","op":"has","val":{"name":"id","op":"eq","val":"13450"}},{"name":"needs_review","op":"eq","val":true}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 2.79 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"user","op":"has","val":{"name":"username","op":"not_in","val":["acfrbot","hdoi5324"]}},{"name":"label","op":"has","val":{"name":"id","op":"eq","val":"13450"}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 4.25 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"user","op":"has","val":{"name":"username","op":"in","val":["hdoi5324"]}},{"name":"label","op":"has","val":{"name":"id","op":"eq","val":"13450"}},{"name":"needs_review","op":"eq","val":false},{"name":"annotation_set_id","op":"in","val":[14174]}]}&page=1&results_per_page=1

In [4]:
annotation_list = objs
unique_annotations = set([a['point']['media_id'] for a in annotation_list])

In [5]:
# Find duplication annotations on media items
from collections import defaultdict
ann_dict = defaultdict(list)
keys = set()
for a in objs:
    media_id = a['point']['media_id']
    is_point = a['point'].get('x') is not None
    if is_point:
        ann_dict[media_id].append(a)

# Iterate through and create list of oldest annotation.
new_annotation_list = []
for media_id, a_list in ann_dict.items():
    oldest_ann_id = min([a['id'] for a in a_list]) 
    oldest_ann = [a for a in a_list if a['id'] == oldest_ann_id]
    new_annotation_list.append(oldest_ann[0])
    
annotation_list = new_annotation_list
media_ids = ann_dict.keys()
objs = new_annotation_list


### Convert into dataframe and save

In [6]:
import pandas as pd
output = []
annotation_set_dir = {}
top_col_names = ['annotation_set_id', 'color', 'comment', 'created_at', 'data', 'id', 'label', 'likelihood', 'needs_review', 'object_id', 'parent_id', 'point', 'suggested_tags', 'tags', 'timestamp', 'updated_at', 'user']
cols_to_ignore = ['data', 'label', 'point', 'tags', 'suggested_tags', 'user', 'color', 'comment']

top_col_names = [item for item in top_col_names if item not in cols_to_ignore ]
label_col_names = ['id', 'label_scheme_id', 'name']
point_col_names = ['id', 'media_id', 't', 'x', 'y']
media_col_names = ['deployment_id', 'key', 'path_best', 'path_best_thm']
pose_col_names = ['alt', 'dep', 'lat', 'lon', 'timestamp']
for r in objs:
    # Ignore annotations with no point
    if r['point']['x'] is not None:
        out = []
        for top_cn in top_col_names:
            out.append(r[top_cn])
        for label_cn in label_col_names:
            out.append(r['label'].get(label_cn))
        for point_cn in point_col_names:
            out.append(r['point'].get(point_cn))    
        media_obj = sq_annotator.sqapi.get(f"/api/media/{r['point']['media_id']}").execute().json()
        for media_cn in media_col_names:
            out.append(media_obj.get(media_cn))
        for pose_cn in pose_col_names:
            out.append(media_obj["pose"].get(pose_cn))
        out.append(r['user']['username'])
        out.append(media_obj['deployment']['timestamp'])
        out.append(media_obj['deployment']['campaign']['name'])
        out.append(media_obj['deployment']['name'])
        
        # Annotation set details
        annotation_set = annotation_set_dir.get(r['annotation_set_id'], None)
        if annotation_set is None:
            annotation_set = sq_annotator.sqapi.get(f"/api/annotation_set", [dict(name="id", op="eq", val= r['annotation_set_id'])]).execute().json()
            if len(annotation_set['objects']) == 1:
                annotation_set = annotation_set['objects'][0]
                annotation_set_dir[r['annotation_set_id']] = annotation_set    
            else:
                print(f"Can't find annotation set {r['annotation_set_id']}")
                continue
        annotation_set_name = annotation_set['name']
        if annotation_set['parent_id'] is not None:
            parent_annotation_set = annotation_set_dir.get(annotation_set['parent_id'], None)
            if parent_annotation_set is None:
                parent_annotation_set = sq_annotator.sqapi.get(f"/api/annotation_set/{annotation_set['parent_id']}").execute().json()
                annotation_set_dir[annotation_set['parent_id']] = parent_annotation_set
            annotation_set_name = f"{annotation_set_name} ({parent_annotation_set['name']})"
            
        out.append(annotation_set_name)
        out.append(annotation_set['media_collection']['name'])
        output.append(out)

label_col_names[0] = 'label_id'
point_col_names[0] = 'point_id'
column_names = top_col_names+label_col_names+point_col_names+media_col_names+pose_col_names+['username', 'deployment_timestamp', 'campaign', 'deployment_name', 'annotation_set_name', 'media_collection_name']
df = pd.DataFrame(output, columns=column_names)

HTTP-GET: https://squidle.org/api/media/6904177 ... ✅ | 200 | Done in 0.12 s
HTTP-GET: https://squidle.org/api/annotation_set?q={"filters":[{"name":"id","op":"eq","val":14174}]} ... ✅ | 200 | Done in 0.23 s
HTTP-GET: https://squidle.org/api/media/7655042 ... ✅ | 200 | Done in 0.13 s
HTTP-GET: https://squidle.org/api/annotation_set?q={"filters":[{"name":"id","op":"eq","val":13180}]} ... ✅ | 200 | Done in 0.15 s
HTTP-GET: https://squidle.org/api/media/7655041 ... ✅ | 200 | Done in 0.10 s
HTTP-GET: https://squidle.org/api/annotation_set?q={"filters":[{"name":"id","op":"eq","val":13158}]} ... ✅ | 200 | Done in 0.15 s
HTTP-GET: https://squidle.org/api/annotation_set/10828 ... ✅ | 200 | Done in 0.81 s
HTTP-GET: https://squidle.org/api/media/7655044 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7655045 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7139000 ... ✅ | 200 | Done in 0.13 s
HTTP-GET: https://squidle.org/api/annotation_set?q={"filters":[{

In [7]:
# Generate thumbnails and store
import os
from PIL import Image
from datasets.squidle_data import get_bbox_in_pixels, get_bbox_from_point_in_pixels
from sqapi.media import SQMediaObject
import cv2

base_dir = os.path.join(os.getcwd(), "annotation_map")
ann_thumbnail_dir = os.path.join(base_dir, "annotation_thumbnails")
annotations = objs

os.makedirs(base_dir, exist_ok=True)
os.makedirs(ann_thumbnail_dir, exist_ok=True)
no_point_annotations = []
# Loop through annotation and generate thumbnail
for a in annotations:
    # Get image
    m = sq_annotator.sqapi.get(f"/api/media/{a['point']['media_id']}").execute().json()
    ann_thumbnail_file = f"{m['key']}_{a['id']}.jpg"
    if not os.path.exists(os.path.join(ann_thumbnail_dir, ann_thumbnail_file)):
        point = a['point']
        if point['x'] is not None:
            media_url = m.get('path_best')
            media_type = m.get("media_type", {}).get("name")
            mediaobj = SQMediaObject(media_url, media_type=media_type, media_id=m.get('id'))    
            print(f"trying {media_url}")
            orig_image = mediaobj.data()
            img = mediaobj.data(Image.fromarray(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)))  # convert to PIL Image
            polygon_ann = len(point['data'].get('polygon', [])) >= 3
        
            # Bbox annotation
            bbox = None
            if polygon_ann:
                bbox = get_bbox_in_pixels(point['x'],
                                               point['y'],
                                               point['data']['polygon'],
                                               img.size[0],
                                               img.size[1])
                polygon_in_px = [[int((p[0] + point['x']) * img.size[0]), int((p[1] + point['y']) * img.size[1])]
                                 for p in point['data']['polygon']]
            elif not polygon_ann:
                # Point annotation estimation around point.  Rough
                # todo: Change this to some other estimation method
                bbox = get_bbox_from_point_in_pixels(point['x'],
                                                          point['y'],
                                                          img.size[0],
                                                          img.size[1])
            # Crop image
            bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
            ann_thumbnail = img.crop(bbox)
            ann_thumbnail.save(os.path.join(ann_thumbnail_dir, ann_thumbnail_file))
        else:
            no_point_annotations.append(a)

HTTP-GET: https://squidle.org/api/media/6904177 ... ✅ | 200 | Done in 0.10 s
HTTP-GET: https://squidle.org/api/media/7655042 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7655041 ... ✅ | 200 | Done in 0.14 s
HTTP-GET: https://squidle.org/api/media/7655044 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7655045 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7139000 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/7139003 ... ✅ | 200 | Done in 0.10 s
HTTP-GET: https://squidle.org/api/media/2527629 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/6887547 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/6887550 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/6888683 ... ✅ | 200 | Done in 0.12 s
HTTP-GET: https://squidle.org/api/media/6893770 ... ✅ | 200 | Done in 0.11 s
HTTP-GET: https://squidle.org/api/media/6894850 ... ✅ | 200 | Done in 0.11 s

In [31]:
os.path.exists(os.path.join(ann_thumbnail_dir, ann_thumbnail_file))

True

In [9]:
os.path.join(ann_thumbnail_dir, ann_thumbnail_file)

'/home/heather/GitHub/BenthicSynData/source/annotation_map/annotation_thumbnails/PR_20210615_031616_950_LC16_5609001.jpg'

In [53]:
df.to_csv("handfish_detections.csv")

In [9]:
import pandas as pd
import seaborn as sns
import folium
import os
import folium
print( folium.__version__)
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

from folium import IFrame
from folium.plugins import FloatImage
from IPython.display import display
import base64
import branca
#sns.swarmplot(data=df, x="lon", y="lat", hue="deployment_id", palette="bright")


0.17.0


### Generate html file from dataframe

In [12]:
base_dir = os.path.join(os.getcwd(), "annotation_map")
ann_thumbnail_dir = os.path.join(base_dir, "annotation_thumbnails")
centre = ((df.lat.max() + df.lat.min())/2, (df.lon.max() + df.lon.min())/2)
map = folium.Map(location=centre, zoom_start=8, control_scale=True, max_zoom=25, tiles="CartoDB Voyager")
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black',
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black',
]

legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px;
    right: 50px;
    width: 150px;
    height: 100px;
    z-index:9999;
    font-size:14px;
    ">
    <p>Source of Detection</p>
    <p><i class="fa-solid fa-eye"></i>&emsp;Human</p>
    <p><i class="fa-solid fa-cogs"></i>&emsp;AI</p>
</div>
<div style="
    position: fixed;
    bottom: 50px;
    right: 50px;
    width: 150px;
    height: 100px;
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;
    filter: blur(8px);
    -webkit-filter: blur(8px);
    opacity: 0.7;
    ">
</div>
{% endmacro %}
"""

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
map.get_root().add_child(legend)

g_idx = 0
cogs_data, eye_data = [], []

#             <img src=\"""" + row.path_best_thm + """\" alt=\"""" + row.key + """\"> <br>

for g in df.groupby("deployment_name"):

    marker_cluster = MarkerCluster().add_to(map)

    #group_1 = folium.FeatureGroup(g[0]).add_to(map)
    for i, row in g[1].iterrows():
        point = (row.lat, row.lon)
        html = """
<!DOCTYPE html>

            Depth (m): """ + f"{row.dep:.2f}" + """  <br>
            User: """ + row.username + """ <br>
            <img src=\"""" + f"https://annotation-map.s3.ap-southeast-2.amazonaws.com/annotation_thumbnails/{row.key}_{row.id}.jpg" + """\" alt=\"""" + row.key + """\" width="200"> <br>
            Deployment Timestamp: """ + row.deployment_timestamp + """ <br>
            Deployment:  """ + row.deployment_name + """ <br>
            Campaign:  """ + row.campaign + """ <br>
            Annotation Set: """ + row.annotation_set_name + """ <br>
            Media Collection: """ + row.media_collection_name + """ <br>
            <a href=\"""" + row.path_best + """\">Full image """ + row.key + """</a> <br>
            """
   #html = html_template(row.dep, row.username, row.deployment_timestamp, row.path_best_thm, row.key, row.id, row.id )
        
        #print(html)
        html = folium.Html(html, script=True)
        #iframe = branca.element.IFrame(html=html, width=500, height=300)
        popup = folium.Popup(html, max_width=300)

        # icons from https://fontawesome.com/v4/icons/
        if row.username in ['acfrbot', 'hdoi5324']:
            icon = "cogs"
            cogs_data.append([row.lat, row.lon, 1])
        else:
            icon = "eye"
            eye_data.append([row.lat, row.lon, 1])

        #icon = "cogs" if row.username in ['acfrbot', 'hdoi5324'] else "eye"
        folium.Marker(point, icon=folium.Icon(colors[g_idx], icon=icon, prefix="fa"), tooltip=f"{row.deployment_name},\n {row.campaign}", popup=popup, lazy=True).add_to(marker_cluster)
        
    g_idx += 1
#folium.LayerControl().add_to(map)

#display(map)
#HeatMap(eye_data).add_to(map)
map.save(os.path.join(base_dir, "handfish_detection_map.html"))


In [2]:
# Test case
# 
import folium
print( folium.__version__)
import branca
print(branca.__version__)


map = folium.Map(location=[-42., 147], zoom_start=7)
point = (-42., 147.)
id = 8800753
html = """
<!DOCTYPE html>
<html>
            <p>
            Link to annotation thumbnails at https://squidle.org/iframe/api/annotation?q=%7B%22filters%22:[%7B%22name%22:%22id%22,%22op%22:%22eq%22,%22val%22:""" + str(id) + """%7D]%7D&disposition=%22inline%22&page=1&results_per_page=10&template=models/annotation/list_thumbnails.html&nologin=true&include_link=false </br> 
            </p>
            <iframe id="myIFrame" src= https://squidle.org/iframe/api/annotation?q=%7B%22filters%22:[%7B%22name%22:%22id%22,%22op%22:%22eq%22,%22val%22:""" + str(id) + """%7D]%7D&disposition=%22inline%22&page=1&results_per_page=10&template=models/annotation/list_thumbnails.html&nologin=true&include_link=false  width=%22500%22 height=%22300%22 frameborder=%220%22>
            </html>
            """
html = folium.Html(html, script=True)
iframe = branca.element.IFrame(html=html, width=500, height=300)
popup = folium.Popup(iframe, max_width=500)

# icons from https://fontawesome.com/v4/icons/
folium.Marker(point, tooltip=f"click here", popup=popup, lazy=True).add_to(map)
map.save("test_handfish.html")


0.16.0
0.7.1


## Tidy up phantom points

In [ ]:
### Config and squidle access setup
acfrbot_userid = 1433
from collections import Counter

In [9]:
config = "squidle_hand_target.yaml"
abs_config_dir=os.path.abspath("../config/dataset")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    opt = compose(config_name=config)
OmegaConf.set_struct(opt, False)

dataset = SquidleData(opt, semi_supervised=True)
sq_annotator = SquidleAnnotator(api_key=opt.api_token, host=opt.url,
                                label_map_file="./datasets/example_coco.json")
sqapi = SQAPI(api_key=opt.annotate_api_token, host=opt.url)
squidle_connection = SquidleConnection(sqapi=sqapi)


/home/heather/.conda/envs/py310_torch/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'squidle_hand_target.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [12]:
# Get annotations for annotation set
ann_set_to_clean = 14146
ann_endpoint = '/api/annotation'
filter_list1 = [dict(name="annotation_set_id", op="in", val=[ann_set_to_clean])]

all_ann = squidle_connection.recursive_get(ann_endpoint, filter_list=filter_list1)
filter_list1 = [dict(name="annotation_set", op="has", val=dict(name="parent_id", op="in", val=[ann_set_to_clean]))]
all_child_ann = squidle_connection.recursive_get(ann_endpoint, filter_list=filter_list1)
# Get annotations for annotation set
endpoint = '/api/point'
filter_list1 = [dict(name="annotation_set_id", op="eq", val=ann_set_to_clean)]
all_point = squidle_connection.recursive_get(endpoint, filter_list=filter_list1)

HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"annotation_set_id","op":"in","val":[14146]}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.94 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/annotation?q={"filters":[{"name":"annotation_set","op":"has","val":{"name":"parent_id","op":"in","val":[14146]}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.43 s
Retrieving page 1 of 0
HTTP-GET: https://squidle.org/api/point?q={"filters":[{"name":"annotation_set_id","op":"eq","val":14146}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 2.73 s
Retrieving page 1 of 1


In [13]:
print(Counter([a['annotation_set_id'] for a in all_ann]))
print(Counter([a['annotation_set_id'] for a in all_child_ann]))
print(Counter([p['annotation_set_id'] for p in all_point]))
print(Counter([a['point']['annotation_set_id'] for a in all_ann]))
print(Counter([a['point']['annotation_set_id'] for a in all_child_ann]))

Counter({14146: 171})
Counter()
Counter({14146: 181})
Counter({14146: 171})
Counter()


In [9]:
point_dict = {p['id']:p for p in all_point}
ann_dict = {a['point']['id']: a for a in all_ann + all_child_ann}

In [11]:
# Delete points with no annotations
phantom_points = list(set(point_dict.keys()).difference(ann_dict.keys()))
for p_id in phantom_points[3:]:
    point = point_dict[p_id]
    if ann_dict.get(p_id, None) is None and len(point['annotations']) == 0:
        response = squidle_connection.delete(f'/api/point/{p_id}').execute()
        print(response)

HTTP-DELETE: https://squidle.org/api/point/9207501 ... ✅ | 204 | Done in 0.09 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207504 ... ✅ | 204 | Done in 0.10 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207505 ... ✅ | 204 | Done in 0.12 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207506 ... ✅ | 204 | Done in 0.13 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207507 ... ✅ | 204 | Done in 0.08 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207509 ... ✅ | 204 | Done in 0.10 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207510 ... ✅ | 204 | Done in 0.11 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207512 ... ✅ | 204 | Done in 0.09 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207517 ... ✅ | 204 | Done in 0.15 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207518 ... ✅ | 204 | Done in 0.14 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207521 ... ✅ | 204 | Done in 0.10 s
b''
HTTP-DELETE: https://squidle.org/api/point/9207522 ... ✅ | 204 | Done in 0.0

In [19]:
# Delete annotations with no label generated by acfrbot
# 
for a in all_ann[1:]:
    if a.get('label', None) is None and a['user']['username'] == 'acfrbot' and a['annotation_set_id'] == ann_set_to_clean:
        a_id = a["id"]
        response = squidle_connection.delete(f'/api/annotation/{a_id}').execute()
        print(response)

HTTP-DELETE: https://squidle.org/api/annotation/9408129 ... ✅ | 204 | Done in 0.09 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9408191 ... ✅ | 204 | Done in 0.12 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9408087 ... ✅ | 204 | Done in 0.11 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9408155 ... ✅ | 204 | Done in 0.15 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9408083 ... ✅ | 204 | Done in 0.12 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420619 ... ✅ | 204 | Done in 0.11 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420631 ... ✅ | 204 | Done in 0.11 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420643 ... ✅ | 204 | Done in 0.14 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420652 ... ✅ | 204 | Done in 0.11 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420658 ... ✅ | 204 | Done in 0.10 s
b''
HTTP-DELETE: https://squidle.org/api/annotation/9420680 ... ✅ | 204 | Done in 0.10 s
b''
HTTP-DELETE: https://

In [62]:
ann_dict.get(p_id, None)

In [51]:
point_endpoint = '/api/point'
filter_list2 = [dict(name="annotation_set", op="has", val=dict(name="user_id", op="in", val="451")),
                dict(name="annotations", op="any", val=(dict(name="annotation_set_id", op="eq", val=ann_set_to_clean)))]
test_point_2 = squidle_connection.recursive_get(point_endpoint, filter_list2)

HTTP-GET: https://squidle.org/api/point?q={"filters":[{"name":"annotation_set","op":"has","val":{"name":"user_id","op":"eq","val":"451"}},{"name":"annotations","op":"any","val":{"name":"annotation_set_id","op":"eq","val":10829}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.09 s
Retrieving page 1 of 0


In [42]:
print([a['id'] for a in test_point_2])
print([a['point']['id'] for a in test_ann])


[9194693, 9194694, 9194696, 9194695, 9194690, 9194691]
[9194690, 9194691, 9194690, 9194695, 9194696, 9194693, 9194694]


In [25]:
print(Counter([a['user']['username'] for a in test_ann]))

Counter({'acfrbot': 5, 'hdoi5324': 2})


In [26]:
print(Counter([a['label']['id'] if a.get('label', None) is not None else 0 for a in test_ann]))
print(Counter([a['point']['annotation_set_id'] for a in test_ann]))



Counter({0: 5, 647: 2})
Counter({14144: 7})


# Media collection with handfish media items



In [3]:
# Open media collection
asi = 13372
media_collection_id = dataset.squidle_connection.sqapi.get(f"/api/annotation_set/{asi}").execute().json()
media_collection_id = media_collection_id['media_collection']['id']
media_endpoint = '/api/media'
filter_list1 = [dict(name="media_collection_media", op="any",
                     val={'name': "media_collection_id", 'op': 'eq', 'val': media_collection_id})]

media = squidle_connection.recursive_get(media_endpoint, filter_list=filter_list1)

# Iterate through media
adjacent_media = []
id_buffer = 3
for m in media:
    # Query for media near media item
    adj_filter = [dict(name="id", op="gt", val=m['id'] - id_buffer),
                  dict(name="id", op="lt", val=m['id'] + id_buffer),
                  dict(name="id", op="!=", val=m['id'])]
    adjacent_media += squidle_connection.recursive_get(media_endpoint, filter_list=adj_filter)



HTTP-GET: https://squidle.org/api/annotation_set/13372 ... ✅ | 200 | Done in 0.88 s
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":11666}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.15 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"id","op":"gt","val":7655038},{"name":"id","op":"lt","val":7655044},{"name":"id","op":"!=","val":7655041}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.11 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"id","op":"gt","val":7652490},{"name":"id","op":"lt","val":7652496},{"name":"id","op":"!=","val":7652493}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.08 s
Retrieving page 1 of 1
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"id","op":"gt","val":7726351},{"name":"id","op":"lt","val":7726357},{"name":"id","op":"!=","val":7726354}]}&pag

In [4]:
og_media_ids = [m['id'] for m in media]
print(len(og_media_ids), len(set(og_media_ids)))
adj_media_ids = [m['id'] for m in adjacent_media if m['id'] not in og_media_ids]
keys = [m['key'] for m in adjacent_media]

42 42


In [ ]:
# Create media_collection
name = "Handfish Test 15pc Dataset - adjacent images"
description = f"Images before and after annotated handfish images. Select {id_buffer} images either side annotated image.  Found {len(adj_media_ids)} additional images from {len(adjacent_media)} adjacent images."
media_collection_id = sq_annotator.create_media_collection(name=name, description=description)
print(media_collection_id)
# Add media
# ADDING MEDIA to collection
sq_annotator.add_media_to_collection(media_collection_id, adj_media_ids)
print("Finished!!!")

HTTP-POST: https://squidle.org/api/media_collection ... ✅ | 201 | Done in 0.35 s
12181
HTTP-GET: https://squidle.org/api/media?q={"filters":[{"name":"media_collection_media","op":"any","val":{"name":"media_collection_id","op":"eq","val":12181}}]}&page=1&results_per_page=1000 ... ✅ | 200 | Done in 0.12 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7655039 ... ✅ | 200 | Done in 0.13 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7655040 ... ✅ | 200 | Done in 0.13 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7726352 ... ✅ | 200 | Done in 0.11 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7726353 ... ✅ | 200 | Done in 0.11 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7726355 ... ✅ | 200 | Done in 0.14 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/7726356 ... ✅ | 200 | Done in 0.11 s
HTTP-POST: https://squidle.org/api/media_collection/12181/media/6972832 ... ✅ | 200 | Don